# Lumber transportation problem (J. Reeb and S. Leavengood)

Millco has three wood mills and is planning three new logging sites. Each mill has a maximum capacity and each logging site can harvest a certain number of truckloads of lumber per day. The cost of a haul is $\$2$/mile of distance. If distances from logging sites to mills are given below, how should the hauls be routed to minimize hauling costs while meeting all demands?

| Logging Site | Mill A | Mill B | Mill C | Max loads per day |
|:------------:|:------:|:------:|:------:|:-----------------:|
| 1            |  8     |  15    |  50    |  20               |
| 2            |  10    |  17    |  20    |  30               |
| 3            |  30    |  26    |  15    |  45               |
| Mill demand  |  30    |  35    |  30    |                   |

### Solution

In [1]:
# install NamesArrays if necessary
using Pkg
Pkg.add("NamedArrays")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed NamedArrays ─── v0.9.6
   Installed Combinatorics ─ v1.0.2
    Updating `~/.julia/environments/v1.8/Project.toml`
  [86f7a689] + NamedArrays v0.9.6
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [861a8166] + Combinatorics v1.0.2
  [86f7a689] + NamedArrays v0.9.6
Precompiling project...
  ✓ Combinatorics
  ✓ NamedArrays
  2 dependencies successfully precompiled in 1 seconds. 203 already precompiled. 1 skipped during auto due to previous errors.


In [3]:
# this solution uses NamedArrays, which are arrays indexed over sets for both x and y dimensions.

using JuMP, HiGHS, NamedArrays

sites = [ 1,  2,  3]
mills = [:A, :B, :C]

cost_per_haul = 4    # don't forget the return trip!

dist = NamedArray( [8 15 50; 10 17 20; 30 26 15], (sites,mills), ("Sites","Mills") )
supply = Dict(zip( sites, [20 30 45] ))
demand = Dict(zip( mills, [30 35 30] ))

m = Model(HiGHS.Optimizer)

@variable(m, x[sites,mills] >= 0)          # x[i,j] is lumber shipped from site i to mill j.

@constraint(m, sup[i in sites], sum(x[i,j] for j in mills) == supply[i] )   # supply constraint
@constraint(m, dem[j in mills], sum(x[i,j] for i in sites) == demand[j] )   # demand constraint

@objective(m, Min, cost_per_haul*sum( x[i,j]*dist[i,j] for i in sites, j in mills ) )      # minimize transportation cost

optimize!(m)

status=termination_status(m)
println(status)

# nicely formatted solution
solution = NamedArray( Int[value(x[i,j]) for i in sites, j in mills], (sites,mills), ("Sites","Mills") )
println( solution )
println()
println("Total cost will be \$", objective_value(m))

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
6 rows, 9 cols, 18 nonzeros
5 rows, 9 cols, 15 nonzeros
Presolve : Reductions: rows 5(-1); columns 9(-0); elements 15(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(155) 0s
          5     5.7600000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  5.7600000000e+03
HiGHS run time      :          0.00
OPTIMAL
3×3 Named Matrix{Int64}
Sites ╲ Mills │ :A  :B  :C
──────────────┼───────────
1             │  0  20   0
2             │ 30   0   0
3             │  0  15  30

Total cost will be $5760.0


In [1]:
dist

LoadError: UndefVarError: dist not defined

### Compact version of the problem

In [4]:
using JuMP, HiGHS, LinearAlgebra

m = Model(HiGHS.Optimizer)

# incidence matrix:
A = [ 1  1  1  0  0  0  0  0  0
      0  0  0  1  1  1  0  0  0
      0  0  0  0  0  0  1  1  1
     -1  0  0 -1  0  0 -1  0  0
      0 -1  0  0 -1  0  0 -1  0
      0  0 -1  0  0 -1  0  0 -1 ]

# supply and demand
b = [ 20, 30, 45, -30, -35, -30 ]

# distances
c = [ 8, 15, 50, 10, 17, 20, 30, 26, 15, ]

@variable(m, x[1:9] >= 0)
@constraint(m, A*x .== b)
@objective(m, Min, 4*dot(c,x))

optimize!(m)
xsol = value.(x)
display( reshape(xsol,3,3)' )

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
6 rows, 9 cols, 18 nonzeros
0; Iter: Time   1.333e-08; average =   1.333e-09; Bound =   1.342e-06
5 rows, 9 cols, 15 nonzeros
Presolve : Reductions: rows 5(-1); columns 9(-0); elements 15(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(155) 0s
          5     5.7600000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  5.7600000000e+03
HiGHS run time      :          0.00


3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.0  20.0   0.0
 30.0   0.0   0.0
  0.0  15.0  30.0